<a href="https://colab.research.google.com/github/MaximL98/Manipulated-Reality/blob/master/video_analysis/colab_notebooks/video_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project Part I**
***This Notebook is the first from three steps, perfom video preprocessing.***

The following code perfom video preprocessing that includes:


*   Face extraction
*   Frame per second reduction
*   Setting maximum video length
*   Resizing images



In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/ManipulatedReality/VideoDatasets/Celeb-DF-V2

Celeb-real  Celeb-synthesis  Fake  Real


In [ ]:
!ls /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2

Celeb-real	       normalized_data_fake   tmp		      val_df_30fps.csv
Celeb-real-30fps       normalized_data_real   train_df_30fps.csv      val_df_30fps.csv.csv
Celeb-synthesis        test_df_30fps.csv      train_df_30fps.csv.csv  val_df.csv
Celeb-synthesis-30fps  test_df_30fps.csv.csv  train_df_30fps.gsheet   val_df.gsheet
feature_extracted      test_df.csv	      train_df.csv


In [ ]:
# Install face_recognition library
!pip install face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=cb62e1d1239ca126d32b5eef9abd2aca2efaae92f8dd9c2a67349121237a649a
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
# Import necessary libraries
import cv2
import os
import numpy as np
import face_recognition
import keras
import pandas as pd
import torchvision.transforms as transforms
from pathlib import Path

In [ ]:
def get_video_paths(folder_path):

  video_paths = [] # List to store found video file paths

  # Recursively traverse the directory tree starting from the given folder_path
  for root, _, files in os.walk(folder_path):
    for file in files:
      # Check if the file extension is one of the supported video formats
      if os.path.splitext(file)[1].lower() in ('.mp4', '.avi', '.mov', '.wmv', '.npy'):  # Common video extensions and npy
        video_path = os.path.join(root, file) # Construct the full path to the video file
        video_paths.append(video_path) # Add the video path to the list
  return video_paths

In [ ]:
def face_detection(image_array, width, height):
    # Locate faces in the frame
    face_locations = face_recognition.face_locations(image_array, model='cnn')
    # Empty numpy array to return in case of not faces were detected
    empty_array = np.array([])
    # Check if face was detected in the frame
    if face_locations:
        top, right, bottom, left = face_locations[0]
        # Save the face frame and resize it
        try:
            face_frame = image_array[top:bottom, left:right]
            resized_face_frame = cv2.resize(face_frame, (width, height), interpolation=cv2.INTER_AREA)
            return resized_face_frame

        except IndexError:
            print("Error: Could not extract face due to invalid bounding box coordinates.")
            return empty_array  # Indicate failure

    else:
        print("No faces detected in the image.")
        return empty_array  # Indicate no faces found

In [ ]:
def extract_video_frame(video_path, video_name):
    # Check if file was all ready processed
    if os.path.exists(video_name + "_processed.npy"):
        print(f"Numpy array for this file {video_name} already exists!")
        return

    # Create a VideoCapture object to read the video
    cap = cv2.VideoCapture(video_path)
    # Check if the video was opened successfully
    if not cap.isOpened():
        print("Error opening video!")
        return None

    # Video properties
    width = 224
    height = 224

    # Empty array to store the frames
    frames = []

    print(f'Starting to extract {video_name} file...')

    error_detect = 0

    # Process each frame of the video
    try:
        while True:
            # Capture frame-by-frame
            ret, frame = cap.read()
            # Target time interval between frames in milliseconds
            target_fps = 15
            # In some of the videos in the real dataset are originaly slowed down by x2
            # To deal with that, x2 fps will be taken
            if '#' in video_name:
                target_fps = 30

            subsample_rate = int(1000 / target_fps)  # Convert FPS to milliseconds
            # Move to the next frame based on the subsample rate
            cap.set(cv2.CAP_PROP_POS_MSEC, (cap.get(cv2.CAP_PROP_POS_MSEC) + subsample_rate))
            # Check if the frame was read correctly
            if not ret:
                print("No more frames to capture!")
                break

            # Convert frame to RGB
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            face_frame = face_detection(frame, width, height)

            if face_frame.size == 0:
                error_detect += 1
                if error_detect > 100:
                    break

            if face_frame.size != 0:
                frames.append(face_frame)

            # Maximum seq length
            if len(frames) == 200:
                break

    finally:
        # Release the capture and close all windows
        cap.release()

    # Save the processed frames as npy array
    if video_name:
        print(f"Saving video frame as numpy array as {video_name}_processed.npy")
        np.save(f"{video_name}_processed.npy", frames)

In [ ]:
import time

# Main function
def main(folder_paths, save_folders):

    # For loop, process both real and fake data
    for index in range(len(folder_paths)):
        # Path to folder which contains data
        folder_path = folder_paths[index]
        # Path to folder in which processed data will be saved
        save_folder = save_folders[index]
        # List of availble types in which the data was split
        types = ["train", "test", "val"]
        for type_ in types:
            # Get paths of all videos
            video_paths = get_video_paths(folder_path + '/' + type_)
            # If such folder does not exist, create it
            save_folder_tmp = save_folder  + '/' + type_
            # Initialize index to follow how many videos finished processing
            i = 0
            # Get total number of files in a folder
            num_files = len(video_paths)
            # For smaller model process only 10-20% of the data, for full model 100%
            num_to_process = int(num_files*1)
            # 100% of data
            ## num_to_process = num_files
            # Iterate each video, extract the frames which include faces
            for video_path in video_paths:
                if i == num_to_process:
                    print(f"Reached set limit of {num_to_process} out of {num_files}")
                    break;
                print(f"Video {i + 1}/{len(video_paths)}")
                # Get video name
                video_name = Path(video_path).stem
                video_name = save_folder_tmp + '/' + video_name
                # Preprocesse the video
                start_time = time.time()
                extract_video_frame(video_path=video_path, video_name=video_name)
                print("--- %s seconds ---" % (time.time() - start_time))
                i+=1




15fps 200 seq length


In [ ]:
folder_paths = ["/content/drive/MyDrive/ManipulatedReality/VideoDatasets/Celeb-DF-V2/Celeb-real",
                "/content/drive/MyDrive/ManipulatedReality/VideoDatasets/Celeb-DF-V2/Celeb-synthesis"]

save_folders = ["/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-real",
                "/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis"]

In [ ]:
main(folder_paths,save_folders)

Streaming output truncated to the last 5000 lines.
Numpy array for this file /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/test/id17_id20_0000 already exists!
--- 0.0005486011505126953 seconds ---
Video 657/1131
Numpy array for this file /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/test/id23_id2_0006 already exists!
--- 0.0006885528564453125 seconds ---
Video 658/1131
Numpy array for this file /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/test/id25_id22_0008 already exists!
--- 0.0001761913299560547 seconds ---
Video 659/1131
Numpy array for this file /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/test/id33_id37_0007 already exists!
--- 0.0006990432739257812 seconds ---
Video 660/1131
Numpy array for this file /content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synt

In [ ]:
# Function to label data, after it was splitted
# If data splitting was made not using "split_train_test_data" then this function is needed to label data
def label_data(real_train_folder, real_test_folder, real_val_folder,
              fake_train_folder, fake_test_folder, fake_val_folder):

    # Lists to store file paths and labels
    real_train_paths = get_video_paths(real_train_folder)
    real_test_paths = get_video_paths(real_test_folder)
    real_val_paths = get_video_paths(real_val_folder)

    fake_train_paths = get_video_paths(fake_train_folder)
    fake_test_paths = get_video_paths(fake_test_folder)
    fake_val_paths = get_video_paths(fake_val_folder)

    # Assign labels (1 for real, 0 for fake)
    real_label = 1
    fake_label = 0

    # Create DataFrames for real videos
    train_df_real = pd.DataFrame({"video_path": real_train_paths, "label": [real_label] * len(real_train_paths)})
    test_df_real = pd.DataFrame({"video_path": real_test_paths, "label": [real_label] * len(real_test_paths)})
    val_df_real = pd.DataFrame({"video_path": real_val_paths, "label": [real_label] * len(real_val_paths)})

    # Create DataFrames for fake videos
    train_df_fake = pd.DataFrame({"video_path": fake_train_paths, "label": [fake_label] * len(fake_train_paths)})
    test_df_fake = pd.DataFrame({"video_path": fake_test_paths, "label": [fake_label] * len(fake_test_paths)})
    val_df_fake = pd.DataFrame({"video_path": fake_val_paths, "label": [fake_label] * len(fake_val_paths)})

    # Combine real and fake DataFrames for each split
    train_df = pd.concat([train_df_real, train_df_fake], ignore_index=True)
    test_df = pd.concat([test_df_real, test_df_fake], ignore_index=True)
    val_df = pd.concat([val_df_real, val_df_fake], ignore_index=True)

    # Check if DataFrames were converted to csv already, comment does lines if want to update csv's
    if os.path.exists(paths_to_csv['train_df']+'.csv') and os.path.exists(paths_to_csv['test_df']+'.csv') and os.path.exists(paths_to_csv['val_df']+'.csv'):
        print("DataFrames already converted to csv's files...")
        # Return DataFrames of train, test and valuation
        return train_df, test_df, val_df

    # Save DataFrame as csv file
    train_df.to_csv(paths_to_csv['train_df']+'.csv', sep=',', index=False, encoding='utf-8')
    test_df.to_csv(paths_to_csv['test_df']+'.csv', sep=',', index=False, encoding='utf-8')
    val_df.to_csv(paths_to_csv['val_df']+'.csv', sep=',', index=False, encoding='utf-8')

    # Return DataFrames of train, test and valuation
    return train_df, test_df, val_df

In [ ]:
real_video_folder_train = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-real/train/'
real_video_folder_test = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-real/test/'
real_video_folder_val = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-real/val/'

fake_video_folder_train = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/train'
fake_video_folder_test = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/test/'
fake_video_folder_val = '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/Celeb-synthesis/val/'

In [ ]:
paths_to_csv = {
    'train_df': '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/train_df_full',
    'test_df': '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/test_df_full',
    'val_df': '/content/drive/MyDrive/ManipulatedReality/NumpyConvertedDatabases/Celeb-DF-v2/val_df_full'
}

In [ ]:
# Label the data
label_data(real_train_folder=real_video_folder_train, real_test_folder=real_video_folder_test, real_val_folder=real_video_folder_val,
           fake_train_folder=fake_video_folder_train, fake_test_folder=fake_video_folder_test, fake_val_folder=fake_video_folder_val)

(                                             video_path  label
 0     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 1     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 2     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 3     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 4     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 ...                                                 ...    ...
 8944  /content/drive/MyDrive/ManipulatedReality/Nump...      0
 8945  /content/drive/MyDrive/ManipulatedReality/Nump...      0
 8946  /content/drive/MyDrive/ManipulatedReality/Nump...      0
 8947  /content/drive/MyDrive/ManipulatedReality/Nump...      0
 8948  /content/drive/MyDrive/ManipulatedReality/Nump...      0
 
 [8949 rows x 2 columns],
                                              video_path  label
 0     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 1     /content/drive/MyDrive/ManipulatedReality/Nump...      1
 2     /cont